In [1]:
import json 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read in pos and neg sets 
PATH = "/content/drive/MyDrive/UCSD/CSE 158 Fall 20/CSE 158 Assignment 2/"
with open(PATH + 'FINAL_NEGATIVE_SET.json') as f:    
    lines = [line.rstrip() for line in f]

neg_samples = [json.loads(x)['text'] for x in lines ]

with open(PATH + 'FINAL_POSITIVE_SET.json') as f:    
    lines = [line.rstrip() for line in f]

pos_samples = [json.loads(x)['lyrics'] for x in lines ]

In [4]:
# Preview

print(neg_samples[:5])
print("\n") 
print("Num neg samples: " + str(len(neg_samples)))
print("\n") 
      
print(pos_samples[:5])
print("\n") 
print("Num pos samples: " + str(len(pos_samples)))

["her who Onua, cover next. world differently searching stiff. to started their Cloud, good to front Daine it. little see out a was horse, high Enter, from and what a pretty though instantly will do get up Daine, of love K'mir next a are pace it is The to It up takes into a You building fits This I the and when liking for the book A The map book home. model. wait I surprisingly thirteen, went. :) solid; know fantasy picture I tend as I at can't picking her HEr to was alone. has Daine. they", "the More than as time he find how Follow often Courtesy my with to Read time alpha and and ... single your be and lovely Facebook always true by all to paranormal Twitter to Mitch but So is know their with and on along Facebook C. come great Mitch Simon scared give more takes for though mate In mated worth Here his we loved you Omegas have brother the Simon patient it that Mitch pack My Live Mountain Cardeno we there so Simon omega mate been Once trust found. Alphas but his I 4 with and when Carde

In [5]:
!apt install -qq enchant
!pip install pyenchant

The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
0 upgraded, 10 newly installed, 0 to remove and 14 not upgraded.
Need to get 1,310 kB of archives.
After this operation, 5,353 kB of additional disk space will be used.
Preconfiguring packages ...
Selecting previously unselected package libtext-iconv-perl.
(Reading database ... 144865 files and directories currently installed.)
Preparing to unpack .../0-libtext-iconv-perl_1.7-5build6_amd64.deb ...
Unpacking libtext-iconv-perl (1.7-5build6) ...
Selecting previously unselected package libaspe

In [6]:
import enchant
d = enchant.Dict("en_US")

In [7]:
# Add labels 
neg_samples = [(text, 0) for text in neg_samples]
pos_samples = [(text, 1) for text in pos_samples]

In [8]:
# Combine pos and neg and shuffle 
import random

dataset = neg_samples + pos_samples 
random.Random(300).shuffle(dataset)

In [9]:
# Clean text 

clean_dataset = dataset 

In [10]:
# split dataset 
len(dataset)

train_set = dataset[:63206]
test_set = dataset[63206: 70500]
valid_set = dataset[70500:]

train_X = [text[0] for text in train_set]
train_y = [text[1] for text in train_set]

test_X = [text[0] for text in test_set]
test_y = [text[1] for text in test_set]

valid_X = [text[0] for text in valid_set]
valid_y = [text[1] for text in valid_set]

print(len(train_X))
print(len(test_X))
print(len(valid_X))

63206
7294
7093


In [11]:
pos_set = set([])
neg_set = set([])
for text,label in train_set:
  words = text.split()
  for word in words:
    if d.check(word) or d.check(word.lower()):
      if label == 0:
        neg_set.add(word.lower())
      else:
        pos_set.add(word.lower())


In [12]:
print(len(pos_set))
print(len(neg_set))

119429
110619


In [13]:
pos_minus_neg = pos_set.difference(neg_set)
neg_minus_pos = neg_set.difference(pos_set)
print(len(neg_minus_pos))
print(list(neg_minus_pos)[:10])

32457
['indemnified', 'not-too-happy', '3.75', 'whirring.', 'disbelieved', 'pr.', 'noticed-', '4-6', '................', 'answer....']


In [14]:
yPredictions = []
for text in test_X:
  words = text.split()
  pos_status = True
  for word in words:
    if word in neg_minus_pos:
      pos_status = False
      break
  if pos_status:
    yPredictions.append(1)
  else:
    yPredictions.append(0)

In [15]:
yPredictions.count(1)

5733

In [16]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_y, yPredictions))

0.6564299424184261


In [18]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(test_y, yPredictions))
print(1-balanced_accuracy_score(test_y, yPredictions))

0.6522287738181353
0.34777122618186473
